In [2]:
import http.client
import json,time
import requests
import re
import os
from pathlib import Path


In [3]:
INFINI_API = "sk-c7cssl4bkglsrwf2"
INFINI_API_2 = "sk-c7erk6qaqhkz5t72"
INFINI_API_List = ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"]

In [4]:
modelList = [
    'llama-3-70b-instruct',
    'llama-3-8b-instruct',
    'chatglm3',
    'chatglm2-6b',
    'chatglm2-6b-32k',
    'infini-megrez-7b',
    'llama-2-7b-chat',
    'llama-2-13b-chat',
    'llama-2-70b-chat',
    'llama-2-70b',
    'baichuan2-7b-chat',
    'baichuan2-13b-chat',
    'baichuan2-13b-base',
    'chatglm3-6b',
    'chatglm3-6b-32k',
    'chatglm3-6b-base',
    'qwen-7b-chat',
    'qwen-14b-chat',
    'qwen-72b-chat',
    'qwen-72b',
    'qwen1.5-7b-chat',
    'qwen1.5-14b-chat',
    'qwen1.5-72b-chat',
    'qwen1.5-72b',
]
evaluateModelList = ['llama-3-70b-instruct','qwen1.5-72b-chat',]
answerModelList = [    
    'llama-3-8b-instruct',
    # 'chatglm3',
    # 'chatglm2-6b',
    # 'chatglm2-6b-32k',
    'infini-megrez-7b',
    'llama-2-7b-chat',
    'llama-2-13b-chat',
    'llama-2-70b-chat',
    'llama-2-70b',
    'baichuan2-7b-chat',
    'baichuan2-13b-chat',
    'baichuan2-13b-base',
    # 'chatglm3-6b',
    # 'chatglm3-6b-32k',
    # 'chatglm3-6b-base',
    'qwen-7b-chat',
    'qwen-14b-chat',
    'qwen-72b-chat',
    'qwen-72b',
    'qwen1.5-7b-chat',
    'qwen1.5-14b-chat',
    'qwen1.5-72b',
]

In [5]:
knowledgeList = ['Knowledge Memorization','Knowledge Understanding','Knowledge Applying','Knowledge Creating']

# DataProcess

In [6]:
def LLMCompletions(prompt,modelName:str = "infini-megrez-7b",INFINI_API = "sk-c7cssl4bkglsrwf2",returnContent:bool = True,**kwargs):
    url = "https://cloud.infini-ai.com/maas/"+modelName+"/nvidia/chat/completions"
    payload = {
        "model": "string",
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "stream": False,
        "temperature": kwargs["temperature"] if "temperature" in kwargs else 0.7,
        "top_p": kwargs["top_p"] if 'top_p' in kwargs else 1,
        "top_k": kwargs['top_k'] if 'top_k' in kwargs else -1,
        "n": kwargs['n'] if 'n' in kwargs else 1,
        "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else None,
        "stop": kwargs['stop'] if 'stop' in kwargs else None,
        "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else 0,
        "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else 0
    }
    index = 0
    while index < len(INFINI_API_List):
        headers = {
                'Content-Type': "application/json",
                'Accept': "*/*",
                'Authorization': "Bearer "+INFINI_API_List[index],
        } 
        response = requests.post(url, json=payload, headers=headers,)
        if response.status_code == 200:
            response.encoding = 'utf-8'
            data = response.json()
            content = data['choices'][0]['message']['content']
            content = content.replace(',\n}','\n}')
            if returnContent:
                return content
            try:
                content = json.loads(content)
            except:
                content = content.replace('\n','')
            data['choices'][0]['message']['content'] = content
            
            return json.dumps(data['choices'][0]['message']['content'])

        else:
            print(response.status_code)
            try:
                print(response.json())
            except:
                pass
        index += 1
    return "Cannot connect to the model "+modelName

In [7]:
data_format = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'context':None,
    'retrieval_context':None,
}

In [8]:
data_format_ins = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'context':None,
    'retrieval_context':None,
    'time':-1
}
def get_LLM_Reply(filepath,savePath,errorPath,fileName = None,):
    with open(filepath) as f:
        data = json.load(f)
    instructions = data['adapter_spec']['instructions']
    questionList = data['request_states']
    errorItem = []
    if not fileName:
        fileName = Path(filepath).name
    save = {'fileName':fileName,'class':knowledgeList[int(fileName[0])-1],'data':[]}
    for index,item in enumerate(questionList):
        if 'id' in item['instance']:
            data_format_ins['id'] = item['instance']['id']
        else:
            data_format_ins['id'] = fileName+'%04d'% index
        prompt = instructions+'\n'+item['instance']['input']['text']
        data_format_ins['input'] = prompt
        print(prompt)
        if item['instance']['references'][0]['tags'][0] == 'correct':
            data_format_ins['expected_output'] = item['instance']['references'][0]['output']['text']
        else:
            data_format_ins['expected_output'] = None
        for model in answerModelList:
            data_format_ins['AnswerModel'] = model
            print(model)
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model)
            end = time.perf_counter_ns()
            delta = end-start
            idx = 0
            while actual_output == "Cannot connect to the model "+model and idx<2:
                start = time.perf_counter_ns()
                actual_output =  LLMCompletions(prompt,modelName=model)
                end = time.perf_counter_ns()
                delta = end-start
                idx += 1
            if actual_output == "Cannot connect to the model "+model:
                
                errorItem.append({'fileName':fileName,'id':data_format_ins['id'],"AnswerModel":model,"input":prompt,"expected_output":data_format_ins['expected_output']})
                continue
            print(idx,delta,actual_output,sep='\t')
            data_format_ins['actual_output'] = actual_output
            data_format_ins['time'] = delta
            save['data'].append(data_format_ins.copy())
            print('*'*70)
        print('+'*70)
    errorItemFinal = []
    while errorItem:
        item = errorItem.pop()
        data_format_ins['id'] = item['id']
        model = item['AnswerModel']
        data_format_ins['AnswerModel'] = model
        prompt = item['input']
        data_format_ins['input'] = prompt
        data_format_ins['expected_output'] = item['expected_output']
        start = time.perf_counter_ns()
        actual_output =  LLMCompletions(prompt,modelName=model)
        end = time.perf_counter_ns()
        delta = end-start
        idx = 0
        while actual_output == "Cannot connect to the model "+model and idx<2:
            print('\t'+str(idx)+'\ttest')
            start = time.perf_counter_ns()
            actual_output =  LLMCompletions(prompt,modelName=model)
            end = time.perf_counter_ns()
            delta = end-start
            idx += 1
        if actual_output == "Cannot connect to the model "+model:
            errorItemFinal.append(item)
            print("[error]:\t"+str(errorItemFinal[-1]))
            continue
        print(idx,delta,actual_output,sep='\t')
        data_format_ins['actual_output'] = actual_output
        data_format_ins['time'] = delta
        save['data'].append(data_format_ins.copy())
    with open(savePath,'w',encoding='utf-8') as out:
        json.dump(save,out)
    if errorItemFinal:
        error = {'fileName':fileName,'class':knowledgeList[int(fileName[0])-1],'data':[]}
        for i in errorItemFinal:
            error['data'].append(i)
        with open(errorPath,'w',encoding='utf-8') as out:
            json.dump(error,out)
    print('='*70)

In [9]:
for dirName,subDirName,fileNames in os.walk('E:\\Repository\\KoLA\\Sample_Data'):
    print(fileNames)

['1-1_2_high_freq_ent_sample.json', '1-2_1_low_freq_ent_sample.json', '1-3_r_1_simple_sample_sample.json', '2-1_COPEN++csj_sample.json', '2-2_COPEN++cpj_sample.json', '2-3_COPEN++cic_sample.json', '2-4_FewNERD++inter_sample.json', '2-4_FewNERD++intra_sample.json', '2-4_FewNERD++supervised_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json', '2-7_MAVEN-ERE_sample.json', '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json', '4-1_without_triples_sample.json', '4-1_with_triples_sample.json', '4-2_r_without_triples_sample.json', '4-2_r_with_triples_sample.json', 'README.md']


In [10]:
with open('./data/4-2_r_with_triples_sample.json') as f:
    data_view = json.load(f)
len(data_view['data'])

80

In [11]:
fileNames =['1-1_2_high_freq_ent_sample.json', '1-2_1_low_freq_ent_sample.json', '1-3_r_1_simple_sample_sample.json', '2-1_COPEN++csj_sample.json', '2-2_COPEN++cpj_sample.json', '2-3_COPEN++cic_sample.json', '2-4_FewNERD++inter_sample.json', '2-4_FewNERD++intra_sample.json', '2-4_FewNERD++supervised_sample.json', '2-5_DocRED_sample.json', '2-6_MAVEN_sample.json', '2-7_MAVEN-ERE_sample.json', '2-8_r_DocRED_sample.json', '3-1_hotpotqa_sample.json', '3-2_2wikimultihopqa_sample.json', '3-3_musique_sample.json', '3-4_kqapro_sample.json', '3-5_KoRC++ood_sample.json', '3-6_r_KoRC++ood_sample.json', '4-1_without_triples_sample.json', '4-1_with_triples_sample.json', '4-2_r_without_triples_sample.json', '4-2_r_with_triples_sample.json', 'README.md']
dirName = 'E:/Repository/KoLA/Sample_Data'


In [12]:
# fileName = '1-1_2_high_freq_ent_sample.json'
# for fileName in fileNames[:5]:
#     get_LLM_Reply(os.path.join(dirName, fileName),'./data/'+fileName,'./data/'+fileName[:-5]+'Error'+'.json',fileName)

In [13]:
# for fileName in fileNames[5:7]:
#     get_LLM_Reply(os.path.join(dirName, fileName),'./data/'+fileName,'./data/'+fileName[:-5]+'Error'+'.json',fileName)

In [14]:
# for dirName,subDirName,fileNames in os.walk('E:\\Repository\\KoLA\\Sample_Data'):
#     regex = re.compile('\.json$')
#     for fileName in fileNames[1:]+[fileNames[0]]:
#         if regex.search(fileName):
#             get_LLM_Reply(os.path.join(dirName,fileName),'./data/'+fileName[:3]+'.json','./data/'+fileName[:3]+'Error'+'.json',fileName)
dirName = 'E:\\Repository\\KoLA\\Sample_Data'            
# fileNames = ['2-4_FewNERD++intra_sample.json','4-1_without_triples_sample.json','4-1_with_triples_sample.json','4-2_r_without_triples_sample.json','4-2_r_with_triples_sample.json',]
# for fileName in fileNames:
#     get_LLM_Reply(os.path.join(dirName,fileName),'./data/'+fileName,'./data/'+fileName[:-5]+'Error'+'.json',fileName)

In [15]:
with open('./MATH/MATH/test/counting_and_probability/0.json') as f:
    data = json.loads(f.read())

In [16]:
with open('./MATH/MATH/train/precalculus/11.json') as f:
    data2 = json.loads(f.read())

In [17]:
data['problem']

'The probability of rain tomorrow is $\\frac{1}{11}$.  What is the probability that it will not rain tomorrow?  Express your answer as a common fraction.'

In [18]:
data2['problem']

'Find the distance between the planes $x - 3y + 3z = 8$ and $2x - 6y + 6z = 2.$'

# EvaluteModel Define

deepeval login --confident-api-key KSPBSFpiD1ZYuuMIBjoTRUrV3VqsbcYmbiW9obh95cI=

In [19]:
import langchain
from langchain.llms.base import LLM
from typing import Any, List, Optional,Dict, Mapping, Union
from deepeval.models.base_model import DeepEvalBaseLLM

d:\Anaconda\envs\vllm\lib\site-packages\deepeval\__init__.py:41: UserWarning: You are using deepeval version 0.21.27, however version 0.21.32 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [20]:
        
metrics_format = {
    'metric_metadata':{
        'metric':None,
        'threshold':0,
        'success':True,
        'score':0.8,
        'reason':'',
        'strictMode': False,
        'evaluationModel': 'CustomLLM',
        'evaluationCost': 0
    },
    'metric_configuration': {
        'threshold': 0.5,
        'evaluation_model': 'CustomLLM',
        'strict_mode': False,
        'include_reason': True
    }
}

data_format = {
    'id':0,
    'AnswerModel':'',
    'input':'',
    'actual_output':'',
    'expected_output':None,
    'context':None,
    'retrieval_context':None,
    'cached_metrics_data':[
        {
            'metric_metadata':{
                'metric':None,
                'threshold':0,
                'success':True,
                'score':0.8,
                'reason':'',
                'strictMode': False,
                'evaluationModel': 'CustomLLM',
                'evaluationCost': 0
            },
            'metric_configuration': {
                'threshold': 0.5,
                'evaluation_model': 'CustomLLM',
                'strict_mode': False,
                'include_reason': True
            }
        },
        metrics_format
    ]
}



In [21]:
path = []

In [76]:
class ChatLLM(LLM):
    @property
    def modelName(self)->str:
        return "qwen1.5-72b-chat"
    @property
    def INFINI_API_List(self)->List[str]:
        return ["sk-c7cssl4bkglsrwf2", "sk-c7erk6qaqhkz5t72","sk-c7etq7veyeie4dn2"]
    @property
    def temperature(self)->float:
        return 0.7
    @property
    def top_p(self)->float:
        return 0.1
    @property
    def top_k(self)->int:
        return -1
    @property
    def n(self)->int:
        return 1
    @property
    def max_tokens(self)->int:
        return None
    @property
    def stop(self)->Optional[List[str]]:
        return None
    @property
    def presence_penalty(self)->float:
        return 0
    @property
    def frequency_penalty(self)->float:
        return 0
    def getHeader(self,index_api):  
        headers = {
            'Content-Type': "application/json",
            'Accept': "*/*",
            'Authorization': "Bearer "+self.INFINI_API_List[index_api%len(self.INFINI_API_List)],
        }
        return headers
    @property
    def _llm_type(self)->str:
        return "ChatLLM"
    @property
    def _identifying_params(self)->Mapping[str,Any]:
        _param_dict = {
            "modelName":self.modelName,
            "INFINI_API":self.getHeader(self.__fields__['index_api'] if 'index_api' in self.__fields__ else 0),
            "stream":bool(self.stream),
            "temperature":self.temperature,
            "top_p":self.top_p,
            "top_k":self.top_k,
            "n":self.n,
            "max_tokens":self.max_tokens,
            "stop":self.stop,
            "presence_penalty":self.presence_penalty,
            "frequency_penalty":self.frequency_penalty,
        }
        return _param_dict

    def _call(self, prompt: str, stop: Optional[List[str]]= None, run_manager= None,**kwargs: Any) -> str:
        url = "https://cloud.infini-ai.com/maas/"+str(self.modelName)+"/nvidia/chat/completions"
        payload = {
            "model": "string",
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False,
            "temperature": kwargs["temperature"] if "temperature" in kwargs else self.temperature,
            "top_p": kwargs["top_p"] if 'top_p' in kwargs else self.top_p,
            "top_k": kwargs['top_k'] if 'top_k' in kwargs else self.top_k,
            "n": kwargs['n'] if 'n' in kwargs else self.n,
            "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens,
            "stop": stop if stop else self.stop,
            "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty,
            "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
        }
        index = 0

        if 'index_api' not in self.__fields__:
            self.__fields__['index_api'] = -1
        index_api = self.__fields__['index_api']+1
        length = len(self.INFINI_API_List)
        while index < length:
            response = requests.post(url, json=payload, headers=self.getHeader(index_api))
            if response.status_code == 200:
                response.encoding = 'utf-8'
                data = response.json()
                # print(data)
                content = data['choices'][0]['message']['content']
                # print(content)
                # path.append(content)
                if isinstance(content,str):    
                    content = content.replace(',\n}','\n}')
                    content = content.replace(']\n}',']}')
                    content = content.replace('\\','\\\\')
                    # path.append(content)
                    flag = False
                    if 'statements' in content:
                        regex = re.compile('\"statements\":\s+\[.*\]\}',re.DOTALL)
                        flag = True
                    elif 'verdicts' in content:
                        regex = re.compile('\"verdicts\":\s+\[.*\]\}',re.DOTALL)
                        flag = True
                    if flag:
                        matchStr =regex.search(content)
                        # print(matchStr)
                        if matchStr:
                            content = '{'+matchStr.group()
                            # path.append(content)
                try:
                    content = json.loads(content)
                    # path.append(content)
                except:
                    pass
                if isinstance(content,str):
                    return content
                data['choices'][0]['message']['content'] = content
                return json.dumps(data['choices'][0]['message']['content'])

            index += 1
            index_api =  (index_api+1)%length
            self.__fields__['index_api'] = index_api
            print(response.status_code)
            try:
                print(response.json())
            except:
                pass
            time.sleep(1)
        return "Cannot connect to the model "+self.modelName
    def setParameter(self,**kwargs):
        self.temperature = kwargs["temperature"] if "temperature" in kwargs else self.temperature
        self.top_p = kwargs['top_p'] if 'top_p' in  kwargs else self.top_p
        self.top_k = kwargs['top_k'] if 'top_k' in  kwargs else self.top_k
        self.n = kwargs['n'] if 'n' in kwargs else self.n
        self.max_tokens = kwargs['max_tokens'] if 'max_tokens' in kwargs else self.max_tokens
        self.stop = kwargs['stop'] if 'stop' in kwargs else self.stop
        self.presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else self.presence_penalty
        self.frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else self.frequency_penalty
    
    

In [77]:
class CustomLLM(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        # print("prompt:\t"+prompt)
        ret = chat_model.invoke(prompt)
        idx = 0
        if ret == "Cannot connect to the model "+self.get_model_name() and idx<2:
            time.sleep(5)
            ret = chat_model.invoke(prompt)
            idx += 1
        # print('*invoke\t'+ret)
        return ret
    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        try:
            return self.model.modelName
        except:
            return "CustomLLM"
# Replace these with real values
custom_model = ChatLLM()

evaluateModel = CustomLLM(model=custom_model)
# print(azure_openai.generate("Write me a joke"))

In [72]:
evaluateModel.get_model_name()

'qwen1.5-72b-chat'

In [51]:
from deepeval.benchmarks import MMLU
from deepeval.benchmarks.tasks import MMLUTask

In [74]:
benchmark = MMLU(tasks=[MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE])
results = benchmark.evaluate(model=evaluateModel )
print(results)

Processing high_school_computer_science:   1%|          | 1/100 [00:00<01:27,  1.13it/s]

*invoke	C


Processing high_school_computer_science:   2%|▏         | 2/100 [00:01<01:27,  1.12it/s]

*invoke	A


Processing high_school_computer_science:   3%|▎         | 3/100 [00:02<01:26,  1.12it/s]

...

*invoke	C
429
{'code': 20013, 'data': None, 'msg': 'Too Many Requests'}


Processing high_school_computer_science:  99%|█████████▉| 99/100 [02:27<00:01,  1.35s/it]

*invoke	D


Processing high_school_computer_science: 100%|██████████| 100/100 [02:28<00:00,  1.49s/it]

*invoke	C
MMLU Task Accuracy (task=high_school_computer_science): 0.78
Overall MMLU Accuracy: 0.78
0.78


In [ ]:
benchmark.load_benchmark_dataset(task=[MMLUTask.HIGH_SCHOOL_COMPUTER_SCIENCE])[0].expected_output

'C'

In [75]:
benchmark.predictions

,Task,Input,Prediction,Correct
0,high_school_computer_science,Let x = 1. What is x << 3 in Python 3?\nA. 1\n...,C,1
1,high_school_computer_science,"In Python 3, which of the following function c...",A,1
2,high_school_computer_science,"A user enters a Web address in a browser, and ...",A,1
3,high_school_computer_science,Digital images are often represented by the re...,D,0
4,high_school_computer_science,A programmer is writing a program that is inte...,B,1
...,...,...,...,...
95,high_school_computer_science,"Consider the following code segment, which use...",C,1
96,high_school_computer_science,A digital photo file contains data representin...,B,1
97,high_school_computer_science,"In Python 3, what is ['a', 'Chemistry', 0, 1][...",C,0
98,high_school_computer_science,Two computers are built by different manufactu...,D,1


In [ ]:
str('"C"')

'"C"'

In [ ]:
for i in range(len(benchmark.predictions)):
    print(benchmark.predictions.iloc[i])

Task                               high_school_computer_science
Input         Let x = 1. What is x << 3 in Python 3?\nA. 1\n...
Prediction                                                  "C"
Correct                                                       0
Name: 0, dtype: object
Task                               high_school_computer_science
Input         In Python 3, which of the following function c...
Prediction                                                  "A"
Correct                                                       0
Name: 1, dtype: object
Task                               high_school_computer_science
Input         A user enters a Web address in a browser, and ...
Prediction                                                  "A"
Correct                                                       0
Name: 2, dtype: object
Task                               high_school_computer_science
Input         Digital images are often represented by the re...
Prediction                         

In [ ]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric,FaithfulnessMetric
from deepeval.test_case import LLMTestCase

# Replace this with the actual output from your LLM application

metric = AnswerRelevancyMetric(
    threshold=0.5,
    model=evaluateModel,
    include_reason=True
)
metric2 = FaithfulnessMetric(
    threshold=0.5,
    model=evaluateModel,
    include_reason=True,
    # strict_mode=True
)
# test_case = LLMTestCase(
#     input="What if these shoes don't fit?",
#     actual_output=actual_output
# )
test_case = LLMTestCase(
    input=data['problem'],
    retrieval_context=[data['solution']],
    actual_output=actual_output,
    # context=["All customers are eligible for a 30 day full refund at no extra cost."],
    # retrieval_context=["Only shoes can be refunded."],
    # latency=10.0
)
test_case2 = LLMTestCase(
    input=data2['problem'],
    # expected_output=data2['solution'],
    actual_output=actual_output2,
    # context=["All customers are eligible for a 30 day full refund at no extra cost."],
    retrieval_context=[data2['solution']],
    # latency=10.0
)
dataset = [test_case,test_case2]
# for i in dataset:
#     metric.measure(i)
#     print(metric.__name__)
#     print(i.input)
#     print(i.actual_output)
#     print(metric.score)
#     print(metric.success)
    
#     metric2.measure(i)
#     print(metric2.truths)
#     print(metric2.reason)
#     print(metric2.score)
#     print(metric2.success)


# or evaluate test cases in bulk
evaluate(dataset, [metric,metric2,],run_async=False,)

NameError: name 'actual_output' is not defined

In [ ]:
metric.__name__

In [ ]:
text = '{"statements": [\n        "To find the distance between two points in multivariate calculus, we use the formula for the distance between a point pair (x1, y1), (x2, y2) in n-dimensional space.",\n        "Distance = sqrt[((x2 - x1)^2 + (y2 - y1)^2)]",\n        "Here, we have two univariate polynomials in x and y, and we want to find the distance between the two polynomials in x and y.",\n        "$x - 3y + 3z = 8$",\n        "$2x - 6y + 6z = 2$",\n        "To find the distance, we need to compare the coefficients of the two polynomials.",\n        "The coefficients of the second polynomial with respect to x are -6 and 2.",\n        "The coefficients of the second polynomial with respect to y are -3 and 0.",\n        "So, the distance between the two polynomials is.",\n        "$\\sqrt{(-6)^2 + 2^2} = \\sqrt{36 + 4} = \\sqrt{40} = 2\\sqrt{10}$",\n        "Thus, the distance between the two polynomials is 2√10."\n    ]}'

In [ ]:
print(text)

In [ ]:
json.loads(text.replace('\\','\\\\'))

In [ ]:
with open('./.deepeval-cache.json') as f:
    text = f.read()
json.loads(text)

In [ ]:
tmp

In [ ]:
LLMOutputSave().LLMCompletions(data['problem'])

In [ ]:
import requests

url = "https://cloud.infini-ai.com/maas/qwen-7b-chat/nvidia/chat/completions"

payload = {
    "model": "string",
    "messages": [
        {
            "role": "user",
            "content": "Write me a joke?"
        }
    ],
    "stream": False,
    "temperature": 0.7,
    "top_p": 1,
    "top_k": -1,
    "n": 1,
    "max_tokens": None,
    "stop": None,
    "presence_penalty": 0,
    "frequency_penalty": 0
}
headers = {
    "Content-Type": "application/json",
    "Accept": "*/*",
    "Authorization": "Bearer "+INFINI_API
}

response = requests.post(url, json=payload, headers=headers)

print(response.json())

In [ ]:
questions = ["请围绕一个法律主题，写一篇茹斯汀·特里耶导演2023年的电影《坠落的审判》的2000字影评，其中应包含不超过1/5的剧情梗概。"]

In [ ]:
for i in questions:
    print(i,LLMCompletions(i))

In [ ]:
class ChatLLM(LLM):

    modelName = "infini-megrez-7b"
    INFINI_API = "sk-c7cssl4bkglsrwf2"
    # hostname = "cloud.infini-ai.com"
    temperature = 0.7
    top_p = 1
    top_k = -1
    n = 1
    max_tokens:int = None
    stop:str = None
    presence_penalty = 0
    frequency_penalty = 0
    
    headers = {
            'Content-Type': "application/json",
            'Accept': "*/*",
            'Authorization': "Bearer "+INFINI_API,
    }
    # def __init__(self,modelName:str = "infini-megrez-7b",
    #         INFINI_API:str = "sk-c7cssl4bkglsrwf2",
    #         hostname:str = "cloud.infini-ai.com",
    #         temperature:float = 0.7,
    #         top_p:float = 1,
    #         top_k:int = -1,
    #         n:int = 1,
    #         max_tokens:int = None,
    #         stop = None,
    #         presence_penalty:float = 0,
    #         frequency_penalty:float = 0):

    #     self.modelName = modelName
    #     self.INFINI_API = INFINI_API
    #     self.hostname = hostname
    #     self.temperature = temperature
    #     self.top_p = top_p
    #     self.top_k = top_k
    #     self.n = n
    #     self.max_tokens = max_tokens if max_tokens is not None else 'null'
    #     self.stop = stop if stop is not None else 'null'
    #     self.presence_penalty = presence_penalty
    #     self.frequency_penalty = frequency_penalty
    #     self.headers = {
    #         'Content-Type': "application/json",
    #         'Accept': "*/*",
    #         'Authorization': "Bearer "+INFINI_API,
    #     }
        
    @property
    def _llm_type(self)->str:
        return "chatllm"
    @property
    def _identifying_params(self)->Mapping[str,Any]:
        _param_dict = {
            "modelName":ChatLLM().modelName,
            "INFINI_API":ChatLLM().INFINI_API,
            "stream":bool(ChatLLM().stream),
            "temperature":ChatLLM().temperature,
            "top_p":ChatLLM().top_p,
            "top_k":ChatLLM().top_k,
            "n":ChatLLM().n,
            "max_tokens":ChatLLM().max_tokens,
            "stop":ChatLLM().stop,
            "presence_penalty":ChatLLM().presence_penalty,
            "frequency_penalty":ChatLLM().frequency_penalty,
            
        }
        return _param_dict
    @classmethod  
    def _call(self, prompt: str, stop: Optional[List[str]]= None,  **kwargs: Any) -> str:
        # temperature = kwargs["temperature"] if "temperature" in kwargs else ChatLLM().temperature
        # top_p = kwargs["top_p"] if 'top_p' in kwargs else ChatLLM().top_p
        # top_k = kwargs['top_k'] if 'top_k' in kwargs else ChatLLM().top_k
        # n = kwargs['n'] if 'n' in kwargs else ChatLLM().n
        # max_tokens = kwargs['max_tokens'] if 'max_tokens' in  kwargs else ChatLLM().max_tokens
        # stop = stop if stop else ChatLLM().stop
        # presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else ChatLLM().presence_penalty
        # frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else ChatLLM().frequency_penalty
        # stream = 'false'
        # temperature = 0.7
        # top_p = 1
        # top_k = -1
        # n = 1
        # max_tokens = 'null'
        # stop = 'null'
        # presence_penalty = 0
        # frequency_penalty = 0
        # conn = http.client.HTTPSConnection(ChatLLM().hostname)
        # stream = 'false' if not bool(stream) else 'true'
        # prompt = prompt.replace('"','\\"')
        
        url = "https://cloud.infini-ai.com/maas/"+ChatLLM().modelName+"/nvidia/chat/completions"
        payload = {
            "model": "string",
            "messages": [
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            "stream": False,
            "temperature": kwargs["temperature"] if "temperature" in kwargs else ChatLLM().temperature,
            "top_p": kwargs["top_p"] if 'top_p' in kwargs else ChatLLM().top_p,
            "top_k": kwargs['top_k'] if 'top_k' in kwargs else ChatLLM().top_k,
            "n": kwargs['n'] if 'n' in kwargs else ChatLLM().n,
            "max_tokens": kwargs['max_tokens'] if 'max_tokens' in kwargs else ChatLLM().max_tokens,
            "stop": stop if stop else ChatLLM().stop,
            "presence_penalty": kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else ChatLLM().presence_penalty,
            "frequency_penalty": kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else ChatLLM().frequency_penalty
        }
        response = requests.post(url, json=payload, headers=ChatLLM().headers)
        if response.status_code == 200:
            return response.json()['response']
        else:
            return "请求模型"
        
        # payload = f'{{"model": "string","messages": [{{"role": "user","content":"{prompt}"}}],"stream":{stream},"temperature": {temperature},"top_p": {top_p},"top_k": {top_k},"n":{n},"max_tokens": {max_tokens},"stop": {stop},"presence_penalty": {presence_penalty},"frequency_penalty": {frequency_penalty}}}'
        # print(payload)
        # conn.request("POST","/maas/"+ChatLLM().modelName+"/nvidia/chat/completions",payload.encode('utf-8'),ChatLLM().headers,)
        
        # res = conn.getresponse()
        # if res.status == 200:
        #     data = res.read().decode("utf-8")
        #     return data
        #     # response = json.loads(data)
        #     # return response
        # else:
        #     return "请求模型"
        
        
    def setParameter(self,**kwargs):
        # self.stream = kwargs["stream"] if "stream" in kwargs else ChatLLM().stream
        self.temperature = kwargs["temperature"] if "temperature" in kwargs else ChatLLM().temperature
        self.top_p = kwargs['top_p'] if 'top_p' in  kwargs else ChatLLM().top_p
        self.top_k = kwargs['top_k'] if 'top_k' in  kwargs else ChatLLM().top_k
        self.n = kwargs['n'] if 'n' in kwargs else ChatLLM().n
        self.max_tokens = kwargs['max_tokens'] if 'max_tokens' in kwargs else ChatLLM().max_tokens
        self.stop = kwargs['stop'] if 'stop' in kwargs else ChatLLM().stop
        self.presence_penalty = kwargs["presence_penalty"]  if 'presence_penalty' in kwargs else ChatLLM().presence_penalty
        self.frequency_penalty = kwargs['frequency_penalty'] if 'frequency_penalty' in kwargs else ChatLLM().frequency_penalty